<a href="https://colab.research.google.com/github/PranaliGhaywat/PranaliGhaywat/blob/main/daily%20practice/gensim_wordembedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gensim

In [3]:
import gensim.downloader as api
from gensim.models import Word2Vec

In [18]:
model = api.load("glove-twitter-25")

In [19]:
model['cat']

array([-0.96419 , -0.60978 ,  0.67449 ,  0.35113 ,  0.41317 , -0.21241 ,
        1.3796  ,  0.12854 ,  0.31567 ,  0.66325 ,  0.3391  , -0.18934 ,
       -3.325   , -1.1491  , -0.4129  ,  0.2195  ,  0.8706  , -0.50616 ,
       -0.12781 , -0.066965,  0.065761,  0.43927 ,  0.1758  , -0.56058 ,
        0.13529 ], dtype=float32)

In [20]:
model.most_similar("time")

[('break', 0.9538871645927429),
 ('way', 0.942317545413971),
 ('next', 0.9418106079101562),
 ('take', 0.9326795935630798),
 ('now', 0.9321372509002686),
 ('going', 0.9304863810539246),
 ('maybe', 0.9293893575668335),
 ('up', 0.9285207986831665),
 ('work', 0.9283475875854492),
 ('then', 0.9273959398269653)]

In [21]:
model.most_similar(positive=['king', 'woman'], negative=['man'])

[('meets', 0.8841924071311951),
 ('prince', 0.832163393497467),
 ('queen', 0.8257461190223694),
 ('’s', 0.8174097537994385),
 ('crow', 0.813499391078949),
 ('hunter', 0.8131037950515747),
 ('father', 0.8115834593772888),
 ('soldier', 0.81113600730896),
 ('mercy', 0.8082392811775208),
 ('hero', 0.8082264065742493)]

In [8]:
words = ['news','cat','Dog']
for i in words:
  if i in model:
    print("no")
  else:
    print("yes")

no
no
yes


In [9]:
model.similarity("king","queen")

0.9202422

In [10]:
model.similarity("good","bad")

0.91478705

In [22]:
model.similarity("cat","dog")

0.95908207

In [12]:
model.similarity("man","king")

0.7666068

In [23]:
model.similarity("man","queen")

0.6799366

In [24]:
model.similarity("woman","queen")

0.7844197

In [25]:
model.similarity("table","king")

0.6486342

In [28]:
model.similarity("hero","queen")

0.89708215

In [30]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [31]:
df = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [32]:
def sent_to_vec(sentence,model):
  words = sentence.lower().split()
  word_vec = [model[word] for word in words if word in model]
  if len(word_vec) == 0:
    return np.zeros(25)
  return np.mean(word_vec,axis=0)

df["vector"] = df['reviewText'].apply(lambda x: sent_to_vec(x, model))

In [33]:
df["vector"]

,vector
0,"[-0.24744236, 0.32033202, 0.30493858, -0.16206..."
1,"[-0.043894824, 0.24313454, 0.04291241, -0.1934..."
2,"[0.011513514, 0.24480152, 0.17919382, -0.24242..."
3,"[-0.08047342, 0.34340683, 0.18201543, -0.27186..."
4,"[-0.1378161, 0.26317292, -0.047043532, -0.1434..."
...,...
194434,"[-0.15870835, 0.23214492, 0.2181116, -0.12004,..."
194435,"[-0.15435538, 0.2069884, -0.030883525, -0.2968..."
194436,"[0.02376818, 0.2655131, 0.1388564, -0.11096091..."
194437,"[0.06434987, 0.2732726, 0.10586506, -0.0274334..."


In [40]:
def convert_to_sentiment (rating):
  if rating >= 4:
    return 1
  elif rating <= 3:
    return 0
  else:
    return 0
df["sentiment"] = df["overall"].apply(convert_to_sentiment)

In [41]:
X= np.vstack(df['vector'].values)
y = df['sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

multinomial_nb = GaussianNB()
multinomial_nb.fit(X_train, y_train)
y_pred = multinomial_nb.predict(X_test)
accuracy_score(y_test, y_pred)
#

0.6973873688541452

In [42]:
multinomial_lr = LogisticRegression()
multinomial_lr.fit(X_train, y_train)
y_pred = multinomial_lr.predict(X_test)
accuracy_score(y_test, y_pred)

0.7720633614482617

In [44]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
xgb = XGBClassifier(objective="multi:softmax",num_class = 3)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.21      0.31      9099
           1       0.80      0.95      0.87     29789

    accuracy                           0.78     38888
   macro avg       0.68      0.58      0.59     38888
weighted avg       0.74      0.78      0.74     38888

